In [14]:
#Start with importing the basic libraries for data cleaning python and numpy
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None)

In [16]:
#Read the csv
#You can put r before the  file path, so that python can treat the backslashes as raw and not escape
#Or use double backslashes or use forard slashes
file = r"C:\Users\HP\Desktop\UI\important\Data Science\movie-success-prediction\data\raw\imdb_movies_final.csv"
df =  pd.read_csv(file)
df.head()

,title,year,duration,certification,ratings,rating_votes_count,genre
0,Gekijô-ban Kimetsu no Yaiba Mugen Jô-hen,2025,2h 35m,R,8.6,31K,Action
1,Peacemaker,2022–,TV-MA,TV Series,8.3,166K,Action
2,The Terminal List: Dark Wolf,2025–,TV-MA,TV Series,7.8,8.4K,Action
3,The Naked Gun,2025,1h 25m,PG-13,6.6,61K,Action
4,One Battle After Another,2025,2h 41m,R,NaN,NaN,Action


In [17]:
print(len(df))

40500


In [10]:
#check if there are duplicates, data was scraped from imdb and some might have been repeated.
df.duplicated(subset = "title").sum()

np.int64(40056)